# LivePoultryCNN
> Development of a LivePoultry Image Classifier using CNN

## Revision History
- 2025/05/20 Activity

## Step 1. Install Dependencies
> Install the required third-party libraries.

In [1]:
%pip install -U jupyterlab
%pip install -U notebook
%pip install -U pandas
%pip install -U opencv-python
%pip install -U scikit-learn
%pip install -U scikit-image
%pip install -U matplotlib
%pip install -U pillow
%pip install -U seaborn
%pip install -U tensorflow
%pip install -U tabulate
print("Environment is ready...")

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
   - -------------------------------------- 0.5/11.5 MB 1.9 MB/s eta 0:00:06
   -- ------------------------------------- 0.8/11.5 MB 2.0 MB/s eta 0:00:06
   ---- ----------------------------------- 1.3/11.5 MB 1.9 MB/s eta 0:00:06
   ------- -------------------------------- 2.1/11.5 MB 2.3 MB/s eta 0:00:05
   ------- -------------------------------- 2.1/11.5 MB 2.3 MB/s eta 0:00:05
   ------- -------------------------------- 2.1/11.5 MB 2.3 MB/s eta 0:00:05
   ----------- ---------------------------- 3.4/11.5 MB 2.2 MB/s eta 0:00:04
   ------------- -------------------------- 3.9/11.5 MB 2.3 MB/s eta 0:00:04
   ----------------- ---------------------

ERROR: Could not find a version that satisfies the requirement tensorflow (from versions: none)
ERROR: No matching distribution found for tensorflow


Note: you may need to restart the kernel to use updated packages.
Environment is ready...


## Step 2: Import the Packages
> Import all third-party libraries necessary for the CNN model to execute successfully.

In [2]:
import os
os.environ["TF_ENABLE_ONEDNN_OPTS"] = "0"
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

import glob
import json
import time
import warnings
from datetime import datetime

# Suppress specific warnings
warnings.filterwarnings("ignore", category=RuntimeWarning, message="os.fork()")
warnings.filterwarnings("ignore", category=UserWarning, message="Your `PyDataset` class should call")

# Core libraries
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import (
    Input, Conv2D, MaxPooling2D, BatchNormalization, Dropout, Flatten, Dense
)
from tensorflow.keras.optimizers import Adadelta

from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

# GPU configuration
gpus = tf.config.list_physical_devices("GPU")
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("GPU detected. Running on GPU.")
    except RuntimeError as e:
        print(e)
else:
    print("No GPU detected. Running on CPU.")


ModuleNotFoundError: No module named 'tensorflow'

## Step 3: Load Datasets
> Load and prepare the training, validation, and testing datasets.

In [ ]:
# Constants
DATA_DIR = "data"
IMAGE_SIZE = 224
IMAGE_CHANNELS = 3
BATCH_SIZE = 8

# Load all image filepaths
all_images = glob.glob(f"{DATA_DIR}/*/*.jpg")

# Create DataFrame with filepaths and labels
df = pd.DataFrame({
    "filepath": all_images,
    "label": [os.path.basename(os.path.dirname(p)) for p in all_images]
})

# Split into train/val/test
train_val_df, test_df = train_test_split(
    df,
    test_size=0.1,
    stratify=df["label"],
    shuffle=True
)

train_df, val_df = train_test_split(
    train_val_df,
    test_size=0.2,
    stratify=train_val_df["label"],
    shuffle=True
)

# ImageDataGenerator for training
generator1 = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest"
)

train_data = generator1.flow_from_dataframe(
    train_df,
    x_col="filepath",
    y_col="label",
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=True
)

# Number of classes
NUM_CLASSES = max(train_data.classes) + 1

# ImageDataGenerator for validation and test (no augmentation)
generator2 = ImageDataGenerator(rescale=1./255)

val_data = generator2.flow_from_dataframe(
    val_df,
    x_col="filepath",
    y_col="label",
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=True
)

test_data = generator2.flow_from_dataframe(
    test_df,
    x_col="filepath",
    y_col="label",
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False
)


## Step 4: Define the Architecture
> Define the structure of the CNN for LivePoultry classification.

In [ ]:
# Model definition
model = Sequential()

# Input and convolutional layers
model.add(Input(shape=(IMAGE_SIZE, IMAGE_SIZE, IMAGE_CHANNELS)))
model.add(Conv2D(32, (3, 3), activation="swish", padding="same"))
model.add(Conv2D(32, (3, 3), activation="swish", padding="same"))
model.add(Conv2D(32, (3, 3), activation="swish", padding="same"))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), activation="swish", padding="same"))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.1))

# Fully connected layers
model.add(Flatten())
model.add(Dense(32, activation="swish"))
model.add(Dense(32, activation="swish"))
model.add(Dropout(0.1))

# Output layer
model.add(Dense(NUM_CLASSES, activation="softmax"))

# Compile the model
optimizer = Adadelta(learning_rate=1.0, rho=0.95)
model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])

# Display model architecture
model.summary()


## Step 5: Train the Model
> Feed the training-val dataset to the compiled CNN model.

In [ ]:
# Constants
EPOCHS = 100
MODELS = "models"
ANALYSIS = "analysis"
ARCHITECTURE = "CNN"

# Create directories if they don't exist
os.makedirs(MODELS, exist_ok=True)
os.makedirs(ANALYSIS, exist_ok=True)

# Start training
training_timestamp = int(time.time())
history = model.fit(train_data, validation_data=val_data, epochs=EPOCHS)
training_duration = (int(time.time()) - training_timestamp) / 60

# Save model
model_path = f"{MODELS}/topic.{ARCHITECTURE}_{training_timestamp}.keras"
model.save(model_path)

# Save training metrics
metrics_path = f"{ANALYSIS}/metrics_{training_timestamp}.json"
with open(metrics_path, "w") as f:
    json.dump({
        "loss": history.history["loss"],
        "accuracy": history.history["accuracy"],
        "val_loss": history.history["val_loss"],
        "val_accuracy": history.history["val_accuracy"]
    }, f, indent=4)

# Summary output
print(f"Training completed in {training_duration:.2f} minutes.")
print(f"Metrics saved to '{metrics_path}'")
print(f"Model saved to '{model_path}'")


## Step 6: Generate Training Analysis
**Metrics Definitions**
- Loss is computed based on how far each prediction is from the ground truth, specifically using Categorical Cross-entropy.
- Accuracy is the proportion of correct predictions to the total predictions.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Load training metrics
with open(f"{ANALYSIS}/metrics_{training_timestamp}.json", "r") as f:
    metrics = json.load(f)

# Epochs range
epochs = list(range(1, len(metrics["loss"]) + 1))

# Plot: Loss
plt.figure(figsize=(8, 6))
sns.lineplot(x=epochs, y=metrics["loss"], label="Training Loss", color="blue")
sns.lineplot(x=epochs, y=metrics["val_loss"], label="Validation Loss", color="orange")
plt.title("Training and Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig(f"{ANALYSIS}/loss_plot_{training_timestamp}.png")
plt.show()

# Plot: Accuracy
plt.figure(figsize=(8, 6))
sns.lineplot(x=epochs, y=metrics["accuracy"], label="Training Accuracy", color="green")
sns.lineplot(x=epochs, y=metrics["val_accuracy"], label="Validation Accuracy", color="red")
plt.title("Training and Validation Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig(f"{ANALYSIS}/accuracy_plot_{training_timestamp}.png")
plt.show()

print(f"\n


## Step 7: Test the Model
> Run the model using the test dataset.

In [ ]:
# Start prediction timing
start_time = time.time()
results = model.predict(test_data, verbose=1)
prediction_duration = time.time() - start_time

# Time per image
image_prediction_time = prediction_duration / test_data.samples

# Process predictions
if results.shape[1] == 1:
    # Binary classification
    predictions = (results > 0.5).astype("int32").flatten()
else:
    # Multi-class classification
    predictions = results.argmax(axis=1)

# Output timing info
print(f"Total prediction time: {prediction_duration:.4f} seconds")
print(f"Time per image: {image_prediction_time:.4f} seconds")


## Step 8: Display the Results
> Show the actual classes and predictions.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image

# Loop through images, true labels, and predicted labels
for fp, true, pred in zip(filepaths, true_indices, predictions):
    img = Image.open(fp).resize((224, 224))
    filename = os.path.basename(fp)
    actual = labels[true].replace("-", " ").title()
    predicted = labels[pred].replace("-", " ").title()

    plt.figure(figsize=(2.5, 3))
    plt.imshow(img)
    plt.axis('off')

    # Add text below the image
    text = f"{filename}\nActual: {actual}\nPredicted: {predicted}"
    plt.text(0.5, -0.1, text, fontsize=8, ha="center", va="top", transform=plt.gca().transAxes)

    plt.tight_layout()
    plt.show()


## Step 9: Confusion Matrix with Cohen's Kappa Score Analysis
> The ideal matrix is a left-to-right diagonal; however, the Cohen's Kappa score is calculated to quantify the results.

In [ ]:
from sklearn.metrics import (
    confusion_matrix, ConfusionMatrixDisplay, cohen_kappa_score
)

# Get true labels and class names
true_indices = test_data.classes
class_labels = list(test_data.class_indices.keys())

# Compute confusion matrix and Cohen's Kappa
cm = confusion_matrix(true_indices, predictions)
kappa_score = cohen_kappa_score(true_indices, predictions)

# Display confusion matrix
fig, ax = plt.subplots(figsize=(8, 8))
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_labels)
disp.plot(cmap="magma", ax=ax)
plt.title(f"Confusion Matrix with Cohen's Kappa: {kappa_score:.2f}")
plt.savefig(f"{ANALYSIS}/confusion-matrix_{training_timestamp}.png")
plt.show()

# Landis & Koch (1977) interpretation of Kappa
kappa_scale = {
    (-1.0, 0.00): "Poor agreement",
    (0.00, 0.20): "Slight agreement",
    (0.21, 0.40): "Fair agreement",
    (0.41, 0.60): "Moderate agreement",
    (0.61, 0.80): "Substantial agreement",
    (0.81, 1.00): "Almost perfect agreement"
}

# Interpret Kappa score
for interval, label in kappa_scale.items():
    if interval[0] < kappa_score <= interval[1]:
        print(f"Interpretation of Kappa Score: {label}")
        break


---
End code.